# Automate HCPCS Formating for Essette

Every quarter, the business requests HCPCS and ICD10 Codes to be updated in the master list for **Prism**, **Que**, **Salesforce**, and **Essette**. This script is used to format a file for Essette.

This script will format the file given by the business into a format that is acceptable by Essette.

## Prerequisites
- No prior knowledge of Python is required to run this script. However, that knowledge will help you understand the logic and syntax of the script. Specifically, we will be using the **pandas** library to transpose the file given by the business to a DataFrame, then we will use that DataFrame to create a new DataFrame with the necessary information. We will also be using the **datetime** library specifically for naming conventions of our file.
- You will need to **rename** the file given by the business to **request.csv**. If this step is not done, then this script **will not run** properly.
- You will need to save this Jupyter Notebook and the file **request.csv** in the same directory, e.g. in the same location on your local machine.
- You will need a method to run Jupyter Notebooks on your local machine. This script assumes Anaconda has already been installed to your local machine. If Anaconda is not installed in your local machine, then you will need to install it via https://www.anaconda.com/docs/getting-started/anaconda/install.

## Overview of the Process
This script will read the filename **request.csv** and load it into a DataFrame using the pandas library. 

It will then use that DataFrame to make a new DataFrame with the following schema:
- **StagedForRemoval** = 0 (Note: We are assuming codes are not Staged for Removal. If Removal is needed, then they will need to be changed manually)
- **TenantCode** = IP
- **TypeCode** = CD.Service
- **ParentCode** = HCPCS
- **Code** = the HCPCS Code listed in the file called **request.csv**
- **Description** = the Short Description listed in the file called **request.csv**
- **AlternateDescription** = the Long Description listed in the file called **request.csv**
- **StartDate** = the Code Start Date listed in the file called **request.csv**
- **EndDate** = the Code End Date listed in the file called **request.csv**

In [2]:
# Import Libraries
import pandas as pd
from datetime import datetime

today = datetime.today() # Get Today's Date
formatted_date = today.strftime("%m%d%Y") # Strip Today's Date in the MMDDYYYY Format

In [3]:
# Load the CSV file into a DataFrame
#df = pd.read_csv("Master List 09.11.csv")  # Replace with your actual file name
original_hcpcs = pd.read_csv("master-list-essette-12-22-25.csv") # Original Data
df = original_hcpcs # Copy of Data


In [4]:
# Clean the Data

# Strip White Space
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/var/folders/5w/yvs225552w7fvlmr0czfhyqr0000gn/T/ipykernel_29781/2127561264.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [5]:
# Create a new DataFrame with the desired columns and transformations
hcpcs = pd.DataFrame({

    # StagedForRemoval is 0 for add and updates; it is 1 for removals
    "StagedForRemoval": 0,

    # TenantCode is always IP
    "TenantCode": "IP",

    # The TypeCode will be CD.Diagnosis for ICD-10 and CD.Service for HCPCS
    "TypeCode": "CD.Service", 

    # ParentCode is ICD-10 or HCPCS depending on file
    "ParentCode": "HCPCS", 

    # Obtain the Code from the File
    "Code": df["HCPCS"], 

    # Obtain the Short Description from the File
    "Description": df["Short Description"],

    # Obtain the Long Description from the File
    "AlternateDescription": df["Long Description"],

    # Obtain the Start Date from the File. This Date can be from the Past
    "StartDate": df["Code Start Date"],

    # Obtain the End Date from the File. For codes we add, we use a far-off date. For example, 12/31/2099.
    # For codes we want to remove we use a recent date.
    "EndDate": df["Code End Date"],

    # New Column for October 2025
    "CodeText": None
})

In [6]:
# Clean the Data

# Check the Format of the Master List Date
#df["Code Start Date"]
# Convert Datetime into Desired Format
hcpcs["StartDate"] = pd.to_datetime(hcpcs["StartDate"], format = "%d-%b-%y", errors = "coerce")
hcpcs["StartDate"] = hcpcs["StartDate"].dt.strftime("%m-%d-%Y")

hcpcs["EndDate"] = pd.to_datetime(hcpcs["EndDate"], format = "%d-%b-%y", errors = "coerce")
hcpcs["EndDate"] = hcpcs["EndDate"].dt.strftime("%m-%d-%Y")

# Truncate any AlternateDescription values longer than 300 characters
hcpcs["AlternateDescription"] = hcpcs["AlternateDescription"].astype(str).apply(lambda x: x[:300] if len(x) > 300 else x)
# Check AlternateDescription Column for any rows with more than 300 characters
#too_long = hcpcs[hcpcs["AlternateDescription"].str.len() > 300]
#print(f"Rows exceeding 300 characters: {len(too_long)}")

# If you wish to preview the result, uncomment the next line by removing the preceeding #
#print(new_df.head())

# Begin ICD-10 Section

In [8]:
# Make sure to use Excel to parse the file given by the business into a new file called request
# Read the File
original_icd10_request = pd.read_csv("request.csv")

df = original_icd10_request

In [9]:
# Create a new DataFrame with the desired columns and transformations
icd10 = pd.DataFrame({

    # StagedForRemoval is 0 for add and updates; it is 1 for removals
    "StagedForRemoval": 0,

    # TenantCode is always IP
    "TenantCode": "IP",

    # The TypeCode will be CD.Diagnosis for ICD-10 and CD.Service for HCPCS
    "TypeCode": "CD.Diagnosis", 

    # ParentCode is ICD-10 or HCPCS depending on file
    "ParentCode": "ICD10", 

    # Obtain the Code from the File
    "Code": df["Code"], 

    # Obtain the Short Description from the File
    "Description": df["Description"],

    # Obtain Description
    "AlternateDescription": None,

    # Obtain the Start Date from the File. This Date can be from the Past
    "StartDate": "2025-10-01", # October 1, 2025

    # Obtain the End Date from the File. For codes we add, we use a far-off date. For example, 12/31/2099.
    # For codes we want to remove we use a recent date.
    "EndDate": "2099-12-31", # December 31, 2099

    # New Column for October 2025
    "CodeText": None
})

In [10]:
# Clean the Data
# Add the Decimal Place at the 3rd Position
#icd10["Code"] = icd10["Code"].astype(str).str[:3] + "." + icd10["Code"].astype(str).str[3:]

# Convert Datetime into Desired Format
#icd10["StartDate"] = pd.to_datetime(icd10["StartDate"], format = "%d-%b-%y", errors = "coerce")
#icd10["StartDate"] = icd10["StartDate"].dt.strftime("%m-%d-%Y")

#icd10["EndDate"] = pd.to_datetime(icd10["EndDate"], format = "%d-%b-%Y", errors = "coerce")
#icd10["EndDate"] = icd10["EndDate"].dt.strftime("%m-%d-%Y")

# If you wish to preview the result, uncomment the next line by removing the preceeding #
#print(new_df.head())

In [11]:
# Create the filename
hcpcs_filename = "tIMP_Codes_hcpc_" + formatted_date
icd10_filename = "tIMP_Codes_icd10_" + formatted_date

In [12]:
# Load the Cleaned Data into a New CSV File
hcpcs.to_csv(hcpcs_filename + "with_commas.csv", index=False)
hcpcs.to_csv(hcpcs_filename + ".txt", index=False, sep='|')

icd10.to_csv(icd10_filename + "with_commas.csv", index=False)
icd10.to_csv(icd10_filename + ".txt", index=False, sep='|')